In [3]:
import os
import openai

In [2]:
from openai import OpenAI

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "is it too late to try the course?"}
    ])

In [12]:
response.choices[0].message.content


"It's never too late to start a new course or to continue your education. The best time to start is now! Here are a few things to consider:\n\n1. **Check the Enrollment Period**: Make sure you are still within the enrollment deadlines. Some courses may have specific start dates and deadlines, while others may offer rolling admission.\n \n2. **Prerequisites and Preparation**: Ensure you meet any prerequisites or requisite background knowledge for the course. If you need a refresher, consider reviewing the material beforehand.\n\n3. **Time Commitment**: Assess your current schedule to determine if you can dedicate the necessary time for coursework, assignments, and study.\n\n4. **Resources**: Gather all the necessary materials, such as textbooks, software, and lecture notes, before you start.\n\n5. **Support System**: Reach out to the course instructor or advisor if you have any questions or need guidance on how to catch up if you're starting late.\n\nRemember, lifelong learning is valua